## 이론 설명
- [ML-objective-function](https://dongryul.notion.site/ML-xgboost-objective-function-1cfa91f49fdb804ebf02f05b4c390447?pvs=4)

## load lib

In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ndcg_score
from sklearn.utils.class_weight import compute_class_weight

from typing import Tuple

## load data

In [2]:
x, y = load_iris(return_X_y=True)

In [3]:
load_iris().target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
iris_df = pd.DataFrame(x, columns=load_iris().feature_names)
iris_df['target'] = y

In [5]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
iris_df['target'].nunique()

3

## objective function

In [7]:
class DefaultSoftprob:
    def __init__(self, labels : pd.Series, eps : float = 1e-6):
        self.kRows = labels.shape[0]
        self.kClasses = labels.nunique()
        self.weights = np.ones((labels.shape[0], 1), dtype=float)
        self.eps = 1e-6
        
    def softmax(self, x):
        e = np.exp(x)
        return e / sum(e)
    
    def softprob_obj(self,
                     labels : np.ndarray, 
                     preds : np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        '''
                input : 
                    labels : 학습데이터 target,
                    preds : 예측 값
                output : 
                    grad : loss gradient 값
                    hess : loss hessian 값
        '''

        ## 아래는 label마다 weight를 주고 싶을때 사용하는 코드이다. 
        ## 현재는 data가 없기에 만약 아래처럼 weight를 주고 싶다면 함수를 Class화 하여서 weigfht를 지정한다음 가져다 쓰면 된다.
        # if data.get_weight().size == 0:
        #     # Use 1 as weight if we don't have custom weight.
        #     weights = np.ones((kRows, 1), dtype=float)
        # else:
        #     weights = data.get_weight()

        ## preds의 shape이 kRows, kClasses와 일치해야한다.
        ## 당연히 맞을거라는 가정으로 진행해도 되지만 혹시나 하는 우려에 체크를 하는 것이 좋다.
        assert preds.shape == (self.kRows, self.kClasses)

        ## preds의 각 위치에 따른 grad과 hess를 구해야 하기에 preds.shape에 맞게 배열 구성
        grad = np.zeros((preds.shape), dtype=float)
        hess = np.zeros((preds.shape), dtype=float)

        ## 아래 eps는 hess값이 0 또는 너무 작은값이 되는 것을 방지하기 위함
        ## 왜 방지를 하느냐? -> 논문 수식을 보면 최적화된 어떤 값을 구하기 위한 식에서 hess가 분모값으로 들어가는데 이때 0이거나 너무 작은 값이면 학습의 불안정이 올수도 있기 때문이다.
        
        ## 이제 각 row를 돌면서 각 예측값에 대한 grad와 hess를 구한다.
        for r in range(self.kRows):
            target = labels[r]
            p = self.softmax(preds[r, :])
            for c in range(self.kClasses):
                assert target >= 0 or target <= self.kClasses
                ### weights의 경우 본인의 선택에 따라 주면 된다.(optional)
                ## 아래 식의 경우 손실 함수를 1차 미분한 식
                ## 답일 경우 p - y(labels = 1) 아닐 경우 p
                g = p[c] - 1.0 if c == target else p[c]
                g = g * self.weights[r]
                ## 아래 식의 경우 손실 함수를 2차 미분한 식
                h = max((2.0 * p[c] * (1.0 - p[c]) * self.weights[r]).item(), self.eps)
                grad[r, c] = g
                hess[r, c] = h

        return grad, hess

## train with objective function

In [8]:
train = iris_df.iloc[:, :4]
target = iris_df['target']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train, target, 
                                                    stratify=target, 
                                                    test_size=0.1,  
                                                    shuffle=True, 
                                                    random_state=42)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((135, 4), (15, 4), (135,), (15,))

In [10]:
default_loss = DefaultSoftprob(labels = y_train)

params = {'max_depth': 6,
          'learning_rate': 0.1,
          'n_estimators': 100,
          "eval_metric":["merror", "mlogloss"],
          "objective": default_loss.softprob_obj,
          "early_stopping_rounds": 5,
          "random_state":42}

evals = [(x_test, y_test)]

xgb = XGBClassifier(**params)
xgb.fit(x_train, y_train, eval_set=evals, verbose=10)

[0]	validation_0-merror:0.06667	validation_0-mlogloss:0.98014
[10]	validation_0-merror:0.06667	validation_0-mlogloss:0.44263
[20]	validation_0-merror:0.06667	validation_0-mlogloss:0.32237
[30]	validation_0-merror:0.06667	validation_0-mlogloss:0.31159
[31]	validation_0-merror:0.06667	validation_0-mlogloss:0.31304


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric=['merror', 'mlogloss'],
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None,
              objective=<bound method DefaultSoftprob.softprob_obj of <__main__.DefaultSoftprob object at 0x13ad22e20>>, ...)

## prediction & evaluation

In [11]:
pred = xgb.predict(x_test)

In [12]:
precision = precision_score(y_test, pred, average='micro')
recall = recall_score(y_test, pred, average='micro')
f1 = f1_score(y_test, pred, average='micro')

print(f'precision : {precision}')
print(f'recall : {recall}')
print(f'f1 : {f1}')

precision : 0.9333333333333333
recall : 0.9333333333333333
f1 : 0.9333333333333333
